In [1]:
import re
import getpass
import gmaps
import requests
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
from urllib.parse import urljoin
from IPython.display import clear_output, display
from parse import parse
import yaml
import time
import os
import sys

#adding prognos tools
sys.path.insert(0, "/home/jose-luis/Envs/niva_prognos/PROGNOS/")
from prognos_tools.encrypt import decryptString
from prognos_tools.Basin_fabric2 import Basin


gmapsToken=b'gAAAAABctxJp5wE73qK6U5VieCi0WXNnNex4KxLZHutsESa8fW9v43lLa1Ag0qsxKFjXXo9MBAdvUpqJPW-QmCE0gH_Opf9g4xAG1VaI2WarO_xDZg44VLMCHkd_6O8ofgp8u4VuFBMr'
key = getpass.getpass('Password: ')
apiKey = decryptString(gmapsToken,key)
gmaps.configure(api_key=apiKey)
del key,apiKey

#Creating folder to store temporary files and downloaded data
saveFolder = './DownloadedData/'

args = [saveFolder,]
!!rm -rf {args[0]} && mkdir -p {args[0]}

Password:  ·······


[]

## Querying extent of the basin
### Creating a VM with the geodatabase on it

In [2]:
machineInfo={}
#Vm characteristics
machineInfo['instance'] = 'prognos-basin'
machineInfo['username'] = 'jose-luis'
machineInfo['image'] = 'geonorway'
machineInfo['imageProject'] = 'nivacatchment'
machineInfo['keyDir'] = '/home/jose-luis/.ssh/threddsBasin'
machineInfo['diskSize'] = '200GB'
machineInfo['region'] = 'europe-west2-a'
machineInfo['machineType'] = 'n1-standard-4'
machineInfo['keyFile'] = F'{os.path.join(machineInfo["keyDir"],machineInfo["username"])}'
machineInfo['pubKeyFile'] = F'{machineInfo["keyFile"] + ".pub"}'

#Actually instantiating the virtual machine 
basinGenerator = Basin(machineInfo)
basinGenerator.instantiate(fabfile='../../prognos_tools/fabfile.py')
print("The ip of the google cloud instance is {}".format(basinGenerator.ip))

#Testing connection to instance
basinGenerator.callPopen('gcloud compute ssh {username}@{instance} --zone {region}'.format(
**basinGenerator.machineInfo))

TypeError: __init__() missing 1 required positional argument: 'properties'

### Getting basins extent

In [3]:
#Schema were processing will be done
schema = 'metno'

#Getting basin outlets from yaml file
stationsFile = 'langtjern.yaml'
stations = yaml.safe_load(open(stationsFile))
display(stations)

#Actually processing the basin
geoResultsFolder='./geoResults'
basinGenerator.getBasinLayers(stationsFile,schema,saveFolder=geoResultsFolder)
    
# Now we delete the basinGenerator vm
#basinGenerator.kill()

[{'station': {'station_id': 108,
   'station_name': 'Langtjern',
   'longitude': 9.7266598416,
   'latitude': 60.3724626431,
   'epsg': 4326,
   'buffer': 5000}}]

'Directory should be created'

Gonna create the getBasin.sh file
./geoResults/getBasin.sh
fab -f ../../prognos_tools/fabfile.py runScript:./geoResults/getBasin.sh,True,./geoResults/box.txt
Can't find any collection named 'fabfile'!
Traceback (most recent call last):
  File "/home/jose-luis/Envs/niva_prognos/bin/fab", line 10, in <module>
    sys.exit(program.run())
  File "/home/jose-luis/Envs/niva_prognos/lib/python3.6/site-packages/invoke/program.py", line 352, in run
    self.parse_collection()
  File "/home/jose-luis/Envs/niva_prognos/lib/python3.6/site-packages/invoke/program.py", line 444, in parse_collection
    self.load_collection()
  File "/home/jose-luis/Envs/niva_prognos/lib/python3.6/site-packages/fabric/main.py", line 82, in load_collection
    super(Fab, self).load_collection()
  File "/home/jose-luis/Envs/niva_prognos/lib/python3.6/site-packages/invoke/program.py", line 661, in load_collection
    module, parent = loader.load(coll_name)
  File "/home/jose-luis/Envs/niva_prognos/lib/python3.6/site-pac

### Displaying basins

In [16]:
with open(os.path.join(geoResultsFolder,'gmaps.txt'), 'r') as f:
    lines = f.read().splitlines()
    
fig = gmaps.figure()
fig.add_layer(gmaps.geojson_layer(yaml.safe_load(lines[0])))

lines[2:] = [i.split(';') for i in lines[2:]]
outlets = [{"name": i[0], "area": '{:8.2f}'.format(float(i[3]))} for i in lines[2:]]
locations = [(float(i[2]),float(i[1])) for i in lines[2:]]
info_box_template = """
<dl>
<font color="black">
<dt>Name</dt><dd>{name}</dd>
<dt>Area</dt><dd>{area} km<sup>2</sup></dd>
</font>
</dl>
"""                                                
outlet_info = [info_box_template.format(**outlet) for outlet in outlets]   
marker_layer = gmaps.marker_layer(locations, info_box_content=outlet_info)
fig.add_layer(marker_layer)

fig

['\n<dl>\n<font color="black">\n<dt>Name</dt><dd>Langtjern_utløp</dd>\n<dt>Area</dt><dd>    4.76 km<sup>2</sup></dd>\n</font>\n</dl>\n']

Markers(data_bounds=[(60.3724568497833, 9.72671739752392), (60.372476849783304, 9.72673739752392)], markers=[M…

Figure(layout=FigureLayout(height='420px'))